# **26. LLM Trend Note 2**

## **26-1. 들어가며**

안녕하세요 여러분:) LLM Trend Note2에 오신 걸 환영합니다!<br>

지난 노드에서 우리는 최신 LLM의 흐름을 살펴보면서<br>
Foundation model의 조건과 Emergent Abilities의 특징에 대해 알아보았습니다.<br>
그기저엔 대규모 분산 컴퓨팅 기술과<br>
보다 혁신적인 모델 아키텍쳐 및 학습기법에 대한 고민이<br>
단단한 기초를 이루고 있다는 사실도 함께 말이죠.<br>

이번 노드에서는 RLHF를 간접적으로 구현한 언어모델에 한국어 말뭉치를 학습시켜보도록 하겠습니다.<br>
이른바 KoChatGPT라고 할 수 있겠죠?<br>
KoChatGPT를 구현한 레퍼런스 코드는 고우영님의 깃헙에서 참고했습니다.<br>

그러면 몇 가지 퀴즈를 풀어보면서 함께 워밍업을 해보도록 할까요?<br>
(퀴즈 정답은 깃헙의 리드미를 꼼꼼히 읽어보시면 찾으실 수 있습니다:)

- **KoChatGPT 구현을 위해 사용한 foundation model**
    - [참고 Git](https://github.com/SKT-AI/KoGPT2)

- **KoChatGPT에서 차용한 Reward Model, Proximal Policy Optimization 알고리즘의 출처**
    > CollosalAI
    - [RLHF Training Stage2 - Training reward model :](https://github.com/hpcaitech/ColossalAI/tree/main/applications/Chat#RLHF-training-stage2---training-reward-model)
    - [RLHF Training Stage3 - Training model with reinforcement learning by human feedback :](https://github.com/hpcaitech/ColossalAI/tree/main/applications/Chat#RLHF-training-stage3---training-model-with-reinforcement-learning-by-human-feedback)

다음 스텝부터 본격적으로 코드실습을 진행하기에 앞서<br>
KoChatGPT를 영상으로 소개한 자료도 함께 살펴보도록 하겠습니다.<br>

[Video (실전) ChatGPT - replica 만들기 코드실습](https://youtu.be/Iq8erq62s8c)<br>

**소스코드에 대한 설명**은 40:57부터 1:12:40 구간을 참고하세요.<br>
**SFT**와 **RM** 그리고 **RLHF에 대한 내용을 복습**하고 싶으신 분들은 12:18부터 28:53 구간을 참고하세요.<br>

영상을 다 시청하셨다면,<br>
오늘 노드의 학습목표와 학습내용을 짚어보고<br>
프로젝트를 향해 힘차게 달려가볼까요?

### **학습목표**

1. ChatGPT 구현을 위해 필요한 데이터셋의 종류 및 특징을 설명할 수 있습니다.

2. Initial Model, Reward Model, RLHF Model의 학습 로직을 설명할 수 있습니다.

3. KoChatGPT를 개선해 나만의 ChatGPT를 구현할 수 있습니다..

### **학습내용**

1. Supervised Fine Tuning

2. Reward Model의 ranking algorithm 및 loss fuction 설계 원리

3. 언어모델을 강화학습하기 위한 방법론

### **준비물**

LMS 클라우드 및 로컬 컴퓨터에서 진행할 경우 (우분투 20.04 기준) 아래와 같은 환경 및 라이브러리들이 요구됩니다.

#### **Environment & Libraries**

```shell
#torch와 cuda 버전은 https://pytorch.kr/get-started/previous-versions/ 사이트를 참고해 
#dependency를 맞춰 설치하시면 됩니다.
wget https://developer.download.nvidia.com/compute/cuda/11.6.0/local_installers/cuda_11.6.0_510.39.01_linux.run
sudo sh cuda_11.6.0_510.39.01_linux.run

pip uninstall torch -y
pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

pip install transformers==4.28.0
pip install colossalai==0.2.7

pip install openai
pip install langchain==0.0.113
pip install pandas>=1.4.1

pip install --upgrade accelerate
pip install bitsandbytes
pip install loralib
```

위의 대부분은 컨테이너에 이미 설치가 되어 있지만<br>
별도로 설치가 필요한 라이브러리가 있습니다.<br>

**cloud shell에서 아래와 같이 KochatGPT를 설치해주세요. **

```shell
cd aiffel 
git clone https://github.com/airobotlab/KoChatGPT  
cd KoChatGPT/colossalai_ChatGPT_230319/
pip install .  
```

필수 requirement들이 잘 설치되어 있는지 확인해볼까요?

In [1]:
import torch

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
!pip list | grep transformers # transformers 4.28.0

Torch version:1.12.1
Cuda version: 11.3
transformers                  4.28.0


## **26-2. Base model and Dataset for RLHF**

LLM Trend Note1 노드에서 보았던<br>
FLAN의 Instruction Tuning이나 PaLM의 Prompt Engineering 이 효과를 보기 위해선<br>
언어모델의 입력을 단순한 query 형태로 주기보단<br>
정교한 입력 시퀀스를 설계해야 한다고 배웠습니다.

예컨대, 작업의 지시사항이 담긴 instruction과<br>
실제 모델이 작업 내용이 담긴 input,<br>
그리고 CoT(Chain of thought) 형태의 예시답안 등을 prompt로 주는 식으로 말이죠.

그런데 이렇게 긴 prompt를 입력할 수 있으려면 그만한 모델 capacity가 뒷받침 되어야 합니다.<br>
일단 수백에서 수천개의 token을 입력 벡터로 받아낼 수 있어야 하고,<br>
각 토큰에 대한 충분한 셀프어텐션 연산이 가능한 트랜스포머 모듈이 쌓여야 하죠.<br>
우리가 backbone 모델로 사용할 KoGPT-2의 성능을 잠시 확인해볼까요?

허깅페이스의 transformers를 사용하면 토크나이저와 모델을 간단히 불러올 수 있습니다.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


우리가 사용할 모델의 토크나이저가 입력받아 처리할 수 있는 최대 토큰 수를 확인해봅시다.

In [3]:
tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

kogpt-2는 어떻게 토크나이징을 하는지 잠시 확인해 볼까요?

In [4]:
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

In [5]:
tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

In [6]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


내친 김에 디코딩 성능도 확인해보겠습니다.

In [7]:
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


시퀀스가 반복되어 출력되는군요.<br>
그리디 서치 디코딩시 발견되는 전형적인 현상입니다.<br>

이번엔 빔 서치 디코딩을 사용하고 n-gram 패널티까지 부과해보겠습니다.

In [8]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아니, 그게 무슨 말씀이신지 모르겠습니다만."
"무슨 말씀인지 알 수가 없군요."
아무런 대답도 하지 않은 채 그녀는 고개를 끄덕였다.
"그래, 알았어."
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
그녀가 다시 입을 열었다.
"정말 죄송합니다, 고마워요, 고맙습니다"
"


입력 시퀀스와 별 상관 없어 보이는 긴 문단이 생성됩니다.<br>
그럼에도 생성된 문단은 제법 맥락을 갖춘 듯 보입니다.<br>
하지만 문장 간의 정합성이나 일관성은 다소 떨어지는 부분도 관찰됩니다.<br>
이번엔 샘플링 기법까지 추가해 보겠습니다.

In [9]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 나는 내 발자취를 생각했다."
"그건 그렇지 않소. 그게 내가 알고 있었던 사실이오. 나도 그걸 몰랐다오."
"내가 어떻게 알았겠느냐고. 내게는 무슨 일인지 나도 알 수 있지요. 하지만 나는 그것을 알지 못한다지요."
그러면서 그는 다시 입을 열었다.
그때 갑자기 그녀의 얼굴이 창백해지더니 그의 손에 이끌려 사라져 버렸다.
"어떻게 된 일인가. 아니, 그건 알지도 못했소?"
그는 몸을 일으켰다.
"아직 내가 알 수가 없소.


top_p 샘플링 기법도 사용해보겠습니다.

In [10]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그런데 그게 무슨 말입니까?"
그녀는 고개를 끄덕였다.
"무슨 말씀이십니까, 선생님."
"선생님, 저는 저를 사랑합니다만."
그녀가 고개를 갸웃거리며 물었다.
"어떻게 그런 말씀을 하셨습니까? 선생님은 저에게 너무 많은 것을 주신 것 같습니다만, 선생님이 저희를 너무 사랑하셔서 그런 것입니다"
"저도 선생님을 사랑하고 있습니다. 선생님의 사랑으로 저


최선의 디코딩 방법을 찾기 위해선<br>
빔사이즈와 n-gram 패널티, temperature와 샘플링 인자로<br>
조합할 수 있는 최선의 값을 찾아보는 실험이 필요합니다.<br>
다양한 입력 시퀀스로 실험을 해보면 더 좋겠죠?<br>

베이스라인 모델로 사용한 kogpt-2의 일반적인 성능을 확인해봤으니<br>
구체적인 instruction과 prompting을 사용해 어떻게 디코딩을 해내는지도 확인해보세요.<br>
RLHF를 적용하기 전의 실험값을 정리해보면<br>
KoChatGPT의 성능 개선 여부를 확인하는데 도움이 될 것입니다.

우리가 사용하는 kogpt-2는 오리지널 GPT2의 가장 작은 버전입니다.
하단의 이미지는 [GPT2](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)논문의 table-2 입니다.

![출처 : https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf](../Images/lec_26/1.png)

emergent abilities를 기대할 수 있는 foundation model로는 많이 부족한 감이 있죠.<br>

위에서 살펴본 것처럼<br>
단순한 Causal LM에 불과한 kogpt-2는 생성해 낼 문장의 품질을<br>
디코딩 단계에서 인위적으로 조절해주는 방법 밖에 쓸 수 없습니다.<br>
하지만 RLHF를 kogpt-2에 적용한다면<br>
더 좋은 문장을 생성해내는 방법을 모델이 스스로 학습해낼 수 있으리라 기대해 볼 수 있습니다.<br>
고도의 prompting은 어렵겠지만<br>
현재 상태에서 특정 task에 fine-tuning했을 때보다는 성능이 한층 더 개선될 수 있지 않을까요?<br>

kogpt-2에 RLHF를 적용하기기 위해선<br>
새로운 데이터셋으로 일련의 재학습을 해줘야 합니다.<br>
그럼 각 단계별 모델 구현에 앞서 우리가 사용할 데이터셋을 확인해보도록 하겠습니다.<br>

먼저 SFT를 시도할 initial 모델에 쓸 데이터셋을 살펴보겠습니다.

### **데이터셋 확인**

In [11]:
import json 
data_path_1_SFT = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

다음으로 RM에 사용할 데이터셋을 살펴보겠습니다.

In [12]:
data_path_2_RM = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

마지막으로 PPO 학습에 쓰일 데이터를 살펴보겠습니다.

In [13]:
data_path_3_PPO = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

## **26-3. Supervised Fine-Tuning**

### **SFT**

이번 스텝에서는 kogpt-2를 instruction dataset으로 SFT를 진행해 보겠습니다.<br>

먼저 필요한 라이브러리들을 불러오겠습니다.

In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


다음으로 모델과 토크나이저를 불러오겠습니다.

In [15]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


모델 인퍼런스 단계에서 사용할 prompt 딕셔너리 템플릿과 SFT 데이터셋 클래스를 정의하겠습니다.

In [16]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        data_path_1_SFT = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

참고<br>
- ** SFT_dataset 클래스의 이니셜라이저에서
label[:source_len] = -100 코드의 -100이 의미하는 게 무엇인가요?
해당 코드가 필요한 이유와 그 기능은 무엇인가요?**
    - [CrossEntropy](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [17]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

이제 SFT_dataset 클래스를 사용해 훈련셋을 만들고 data collator 인스턴스를 만들겠습니다.

In [18]:
train_dataset = SFT_dataset(data_path_1_SFT='/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [19]:
# 텐서를 텍스트로 디코딩하는 함수
def decode_tensor_to_text(tokenizer, tensor):
    # 텐서를 CPU로 이동하고, numpy 배열로 변환
    tensor = tensor.cpu().numpy()

    # 패딩 토큰과 -100을 제외한 토큰만 선택
    tokens = []
    for token in tensor:
        if token == tokenizer.pad_token_id or token == -100:  # 패딩 토큰 또는 -100일 경우 루프 종료
            break
        tokens.append(token)

    # 토큰을 텍스트로 디코딩
    decoded_text = tokenizer.decode(tokens, skip_special_tokens=True)
    return decoded_text

# input_ids와 labels 텐서를 디코딩하여 텍스트로 출력
input_text = decode_tensor_to_text(tokenizer, train_dataset.input_ids[0])
output_text = decode_tensor_to_text(tokenizer, train_dataset.labels[0])

print('input :', input_text)
print('output:', output_text)

input : ### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.
output: 


훈련을 위한 마지막 단계로 Training arguments를 사용해 trainer 클래스를 정의하겠습니다.

In [20]:
training_args = TrainingArguments(
    output_dir="aiffel/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

참고
- [Huggingface Trainer() document](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments)

SFT 훈련을 진행해볼까요? (빠르게 학습해보기 위해 1epoch만 돌려보도록 하겠습니다.)

In [21]:
trainer.train()
model.save_pretrained('/aiffel/KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.984100
1000,2.776800
1500,2.687200


loss가 잘 줄어들었나요?<br>
이제 문장 생성 능력을 확인하기 위해<br>
빠르게 허깅페이스의 pipleline 클래스를 사용하여 generator를 만들어보겠습니다.

In [22]:
generator = pipeline('text-generation', model='/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 불고기용 고기의 종류와 양에 대한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기는 쇠고기와 함께 먹는 음식 중 하나입니다. 따라서 불고기를 먹을 수 있는 종류는 다양합니다. 예를 들어, 닭가슴살 스테이크, 오므라이스 샐러드 등이 있습니다.

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 42대 부통령직을 수행했습니다.作)作)은 "리처드 닉슨"이 41대 부통령을 수행한 년도를 가리키는 말입니다.作)는 "리처드 닉슨"이 40대 부통령을 맡았던 년도를 의미합니다.作은 "리처드슨"이 50대 부통령

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아주 샌프란시스코에 위치해 있습니다.子供共和國際空港)이라고 불립니다.子供公和国際空港이라는 뜻입니다.子供空和國際公港이라는 이름을 가진 항공사는 다음과 같습니다.\n\n1. 대한항공

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇으로써 미세먼지 정보를 알 수 없습니다. 미세먼지 예보를 확인해 보시는 것이 좋겠습니다.\n\n미세먼지 예보: 일반적으로 미세먼지는 주로 중국에서 발원하여 중국 전역으로 퍼져나가기 때문에 중국발 미세먼지가 유입될


참고
- [HuggingFace Text Generation strategies](https://huggingface.co/docs/transformers/v4.28.1/en/generation_strategies)

SFT 모델의 성능은 어떤가요?<br>
SFT 단계를 최적화하기 위해선 무엇보다도 instruction dataset의 품질과 initial모델의 언어모델링 성능이 중요합니다.<br>
GPT를 새로 pretrain 하여 언어모델 성능을 도약시키는 일은 우리의 학습목표를 넘어서는 일이니<br>
우선은 데이터셋 전처리를 더 수행하고 최상의 디코딩 전략이 적용된 generator를 설계한다면<br>
더 나은 성능을 기대해 볼 수 있을 것입니다.<br>
하지만 지금은 baseline을 빠르게 돌려보는 게 목적입니다.<br>
이제 다음 단계인 reward modeling으로 넘어가 보도록 하겠습니다.<br>

메모리 관리를 위해 캐시를 비우고 넘어가겠습니다.

In [23]:
torch.cuda.empty_cache()

## **26-4. Reward Model**

이번 스텝에서는 RLHF의 두번째 단계인 Reward model을 설계하고 학습해보겠습니다.
필요한 라이브러리들을 불러와 볼까요?

### **RM**

In [24]:
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

Import된 라이브러리들을 보면 SFT단계에서 사용했던 것과는 다른 모듈들이 몇 개 눈에 띕니다.<br>
우선 chatgpt 폴더 안에 있는 RM 관련 모듈들이 눈에 띄네요.<br>
NaiveStrategy라는 모듈도 있습니다.<br>
원본 깃헙 레포짓에는 multi GPU를 사용해서도 KoChatGPT를 실습해볼 수 있도록 하고 있지만<br>
우리는 single GPU를 사용해야 하는 환경이므로 학습전략을 고정시켜놓기 위해 해당 모듈을 따로 import했습니다.<br>
마지막엔 허깅페이스의 transformers에서 gpt2 모델 관련 모듈들도 사용하고 있네요.

이제 Reward model을 설계해 볼까요?<br>
GPTRM_custom 이라는 이름으로 클래스를 선언하겠습니다.

In [25]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

위 클래스의 원본 코드는 [이 링크](https://github.com/hpcaitech/ColossalAI/blob/2e16f842a9e5b1fb54e7e41070e9d2bb5cd64d7c/applications/ChatGPT/chatgpt/nn/gpt_rm.py)에서 확인하실 수 있습니다.

여기서 주의깊게 봐야할 부분은 이니셜라이저의 value_head = nn.Linear(model.config.n_embd, 1) 코드입니다.

이제 SFT에서와 마찬가지로 사용할 모델과 토크나이저를 불러오겠습니다.<br>
with구문의 NaiveStrategy()는 chatgpt/trainer/strategies 폴더의 base 모듈에서 정의된
Strategy클래스를 상속한 NaiveStrategy클래스입니다.

In [26]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


다음으로 RM을 훈련시킬 때 사용할 ranking dataset을 만들어보겠습니다.

In [27]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


kochatgpt_2_RM.jsonl 은<br>
chatGPT, davinch, ada 세개 모델에 같은 prompt를 주고 얻은 세 답변을<br>
순서대로 good, bad, worst로 간주해<br>
순서를 뒤섞어 completion_0, completion_1, completion_2 세 키에 할당하여 만든 데이터셋입니다.<br>
위와 같이 코드를 짜게 되면 chosen과 resjected에 각각<br>
completion_0, completion_1, completion_2 세개 답변이 가능한 모든 조합으로 들어가게 되어<br>
chosen에 worst 답변이 들어가고<br>
rejected에 good답변이 들어간 데이터도 만들어집니다.<br>

위와 같이 ranking dataset을 만들면 RM의 loss는 어떻게 계산이 되는 걸까요?<br>
RM의 loss function은 pairwiseloss라는 이름으로 설계되어 있습니다.<br>
아래 pairwiseloss 코드를 첨부했습니다.<br>
원본 코드는 chatgpt/models 폴더의 loss.py 를 확인해보세요.

```python
class PairWiseLoss(nn.Module):

    def forward(self, chosen_reward: torch.Tensor, reject_reward: torch.Tensor) -> torch.Tensor:
        probs = torch.sigmoid(chosen_reward - reject_reward)
        log_probs = torch.log(probs)
        loss = -log_probs.mean()
        return loss
```

**Q16. 위 코드블럭에서 probs = torch.sigmoid(chosen_reward - reject_reward) 코드를 찾아보세요.
chosen_reward - reject_reward 식은 어떤 연산을 의미하나요?
loss = -log_probs.mean() 코드는 무엇을 최대화하는 연산으로 해석할 수 있을까요?**

```python
total_data_ranking2chosen = []

for tmp in list_data_dict:
     prompt = tmp['prompt']
     ranking = tmp['ranking']

     for index in range(1, len(ranking)):
         n = ranking[0]
         m = ranking[index]


         data = {
             'prompt': prompt,
             'chosen': tmp['completion_{}'.format(n)],
             'rejected': tmp['completion_{}'.format(m)]
         }

         total_data_ranking2chosen.append(data)
```

**Q17. 위 코드대로 ranking dataset 함수를 수정하게 되면 ranking data가 어떻게 만들어지게 되나요?
둘의 차이를 비교하고 어떤 데이터셋을 사용하는게 더 적절한지 이야기해봅시다.**

이제 완성한 ranking dataset을 shuffle한 후 훈련셋을 만들어보겠습니다.<br>
빠르게 돌려보기 위해 전체 데이터중 일부만 학습하도록 하겠습니다.

In [28]:
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [29]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 859.21it/s]


**Q18. RewardDataset 클래스는 어떤 기능을 수행하나요?
chatgpt/dataset폴더 내의 reward_dataset모듈을 참고해 답해보세요.**

데이터셋이 잘 만들어졌는지 하나를 뽑아 확인해봅시다.

In [30]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


마지막으로 RM을 학습해 보겠습니다.<br>

(SFT 훈련때와 마찬가지로 RM 훈련시 많은 자원이 소모됩니다.<br>
모델 체크포인트를 활용할 수 있으니, 각각의 모델을 더 많은 데이터로 더 오래 훈련하고자 할 시,<br>
커널을 초기화 한 후 재학습을 해보세요.<br>
지금은 빠르게 학습해보기 위해 1epoch만 돌려보도록 하겠습니다)

In [31]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [32]:
trainer.fit(use_lora=0)

model.save_pretrained('aiffel/KoChatGPT/output_2_RM')

Train epoch: 100%|██████████| 1/1 [03:53<00:00, 233.45s/it]


RM 학습이 잘 되었는지 확인해보기 위해 임의의 문장을 입력한 후
적절한 reward score를 출력하는지 살펴보도록 하겠습니다.

In [33]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -0.6


In [34]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: -0.6


In [35]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: -0.4


In [36]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: -0.3


input text가 더 좋아질수록 reward score가 점진적으로 상승하나요?<br>
각 reward score 값이 적절해 보이시나요?<br>
reward score가 음수가 된다는 건 어떤 의미일까요?<br>
그 전에 reward score가 음수도 될 수 있도록 하려면 어떻게 해야 할까요?<br>
RM의 출력인 reward score가 scalar가 되도록 하는 게 왜 중요할까요?<br>

RLHF의 마지막 단계인 PPO 학습을 통해 살펴보도록 하겠습니다.<br>

여기서도 메모리 관리를 위해 한 번더 캐시를 비우고 넘어가겠습니다.

In [37]:
torch.cuda.empty_cache()

## **26-5. Proximal Policy Optimization**

### **PPO**

드디어 RLHF의 마지막 세번째 단계인 PPO를 실습해볼 차례입니다.<br>
사용할 라이브러리들을 불러오도록 하겠습니다.

In [38]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

노드에서 소개하는 KoChatGPT의 경우<br>
PPO에 사용할 actor모델은 1단계 SFT 모델을,<br>
critic모델은 2단계 RM 모델을 사용합니다.<br>

그리고 actor 모델이 critic 모델로부터 피드백을 받아 파라미터를 업데이트 할 때<br>
적절한 페널티를 줄 수 있도록 하는 initial model은<br>
SFT모델을 그대로 freezing 하여 사용합니다.<br>

토크나이저는 pretrain 모델인 kogpt-2의 토크나이저를 그대로 사용해야겠죠?

![KOGPT-2 model structure](../Images/lec_26/2.png)

In [41]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/aiffel/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='/aiffel/aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

모델학습에 사용할 옵티마이저와 모델을 준비합니다.

In [42]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [43]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

PPO 학습에 쓸 데이터를 불러와 토크나이징 해줍니다.

In [44]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [45]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [46]:
len(list_prompt)

12000

PPO는 별도의 PPOTrainer 클래스를 설계하여 학습시켜줘야 합니다.<br>
빠르게 실습해보기 위해 1epoch만 돌려보겠습니다.

In [47]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

위 코드블럭의 원본 코드는 chatgpt/trainer 폴더 내의 ppo.py 모듈에서 확인할 수 있습니다.<br>
PPO는 SFT, RM 보다 훨씬 복잡한 단계로 설계되는 강화학습 알고리즘입니다.<br>
PPO의 loss function은 chatgpt/models 폴더 내의 loss.py 모듈에서<br>
PolicyLoss와 ValueLoss 클래스에 정의되어 있습니다.

참고
- [Fine-tuning the LM with RL](https://gist.github.com/JoaoLages/c6f2dfd13d2484aa8bb0b2d567fbf093#3---fine-tuning-the-lm-with-rl)

PPO에 대해 좀 더 자세히 알고 싶으신 분들은 허깅페이스에서 제공하는 [Deep RL Course](https://huggingface.co/learn/deep-rl-course/unit8/introduction?fw=pt)를 참고하세요.

이제 PPO 학습을 진행하도록 하겠습니다.

In [48]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained('/aiffel/KoChatGPT/output_3_PPO')

Episode [10/10]: 100%|██████████| 3/3 [00:17<00:00,  5.87s/it]


드디어 SFT, RM 그리고 PPO 학습이 모두 완료되었습니다.<br>
RLHF가 적용된 koGPT-2의 생성능력을 확인해볼까요?

In [49]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이므로 현재 판매되는 쇠고기의 등급과 양에 따라 다를 수 있습니다. 하지만 일반적으로 불고기용 쇠고기는 최상위 등급의 쇠고기이며, 등급이 높은 쇠고기도 최상위 등급으로 인정을 받습니다. 따라서, 등급이 높아지면 등급은 더 높은 가격으로 인정받게 되며, 현재 불고기용으로 판매되는 불고기용 쇠고기에 대한 등급이 높아지게 됩니다. 따라서, 실제 해당 쇠고기의 등급과 가격은 상이하지만, 현재 국내 기준으로는 불고기용으로 판매되는 불고기를 찾고자 한다면 해당 등급의 쇠고기의 등급이 높은 등급인 경우에는 불고기로 먹을 수 있는 고기로 먹으실 수 있습니다.子立さん らち ら ら らら\  ら ら ら\ ら  ラ ら ら ら  ら ら ら  ら ら 

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'저는 데이터에 대한 의존도를 가지고 있지 않기 때문에 제임스 닉슨의 부통령직 수행년도에 대해 알지 못합니다. 정확한 정보와 관련해서는 "리처드 닉슨의 47대 부통령직 수행년도를 알아내기 위한 추가적인 자료를 찾을 수 없습니다. 그러나 여러 국가들에서 활동하는 부통령은 여러 나라들에서 활동한 경험이 있을 가능성이 있습니다. 따라서 해당 국가들과의 대화와 상호협력 및 협력을 증진하기 위해 40대 부통령직의 수행년도에 대한 답변을 드리지 않습니다. '리처드 닉슨의 46대 부통령직 수행년도는 알려지지 않았습니다. '리처드 닉슨의 46대 부통령직 수행년도는 명확하지 않습니다. 다른 정보는 찾아볼 수 없습니다.' 죄송합니다. '리처드 닉슨의 43대 부통령직 수행년도는 리처드 닉슨이 39대 부통령직을 수행한 시기와 관련된 것일 가능성이 있습니다. '리처드 닉슨의 39대 부통령직 수행년도는 없습니다. '리처드 닉슨의 41대 부통령직 수행년도는 불분명합니다. '리처드 닉슨의 45대 부통령직 수행년도는 불분명합니다. 

## **26-6. 마무리하며**

최종 모델 학습 결과는 어떠셨나요?<br>
중복 토큰 생성 문제를 비롯해 맥락에서 벗어난 한문이나 영문이 출력되기도 합니다.<br>
kogpt-2에 SFT만 적용했을 때와 비교해보면 큰 차이를 느끼지 못할 수도 있습니다.<br>
각 단계에서 사용되는 데이터셋을 충분히 정제하고, 훈련 사이클을 늘려 정교하게 디코딩한다면<br>
훨씬 나은 성능을 기대해볼 수 있습니다.

RLHF의 진가는 고도로 정제된 instruction dataset와 정교하게 설계된 보상체계로 학습되는 Reward model,<br>
그리고 PPO 학습이 안정적으로 이뤄질 수 있도록 하는 충분한 크기의 foundation model이 뒷받침 되었을 때 발휘될 수 있습니다.

그러나 진짜 인간의 피드백이 반영된 데이터셋을 구축하기란 아주 어려운 일입니다.<br>
첫 단계부터 큰 난관이라 할 수 있죠.<br>
하지만 SFT만으로도 굉장히 훌륭한 언어모델을 만들 수 있습니다.<br>
이준범님이 공개한[koalpaca](https://github.com/Beomi/KoAlpaca) 모델이 대표적인 사례라 할 수 있습니다.

**Q20. koalpaca는 foundation model로 어떤 모델들을 사용했나요?**

**Q21. koalpaca의 레퍼런스 모델인 stanford alpaca가 사용한 모델과 데이터셋 그리고 학습방법은 무엇인가요?**

참고링크 - https://crfm.stanford.edu/2023/03/13/alpaca.html

충분한 컴퓨팅 파워가 마련된 여건이라면 koalpaca모델로 RLHF까지 진행해 볼 수도 있을 것 같습니다.<br>

그 전에 오늘 노드를 기반으로 도전해 볼만한 과제들을 수행해보는 것이 도움이 될 것 같습니다.<br>

그럼 우리 함께 프로젝트 노드로 이동해볼까요?